In [1]:
"""
This example demonstrates the workflow to download a publicly available 
TensorFlow model and convert it to Core ML format using the tfcoreml converter.

We use an open source implementation of DeepSpeech model (https://arxiv.org/abs/1412.5567) 
provided by Mozilla: https://github.com/mozilla/DeepSpeech.
"""

In [2]:
import tfcoreml
import numpy as np
import tensorflow as tf

W0710 16:59:39.237015 4739939648 __init__.py:71] TensorFlow version 1.14.0 detected. Last version known to be fully compatible is 1.13.1 .


In [3]:
# Download TensorFlow implementation of DeepSpeech model from https://github.com/mozilla/DeepSpeech
# wget https://github.com/mozilla/DeepSpeech/releases/download/v0.4.1/deepspeech-0.4.1-models.tar.gz
# tar xvfz deepspeech-0.4.1-models.tar.gz

In [4]:
tfmodel_path = './models/output_graph.pb'  # path to the downloaded model
mlmodel_path = './deep_speech.mlmodel'  # path to save converted Core ML model

# convert model and save to the local directory
model = tfcoreml.convert(
    tf_model_path=tfmodel_path, 
    mlmodel_path=mlmodel_path,
        input_name_shape_dict={
        'input_node': [1, 16, 19, 26],
        'input_lengths': [1],
        'previous_state_h': [1, 2048],
        'previous_state_c': [1, 2048]
    },
    output_feature_names=['logits'],
    use_coreml_3=True
)

# Optionally, we can print and inspect converted Core ML model
# from coremltools.models.neural_network.printer import print_network_spec_coding_style
# print_network_spec_coding_style(model.get_spec())

W0710 16:59:39.820260 4739939648 deprecation_wrapper.py:119] From /Volumes/data/work/coreml/coremltools/coremltools/converters/nnssa/frontend/tensorflow/graphdef_to_ssa.py:21: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0710 16:59:39.821039 4739939648 deprecation_wrapper.py:119] From /Volumes/data/work/coreml/coremltools/coremltools/converters/nnssa/frontend/tensorflow/graphdef_to_ssa.py:22: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.



0 assert nodes deleted


W0710 16:59:41.819582 4739939648 deprecation_wrapper.py:119] From /Volumes/data/work/coreml/coremltools/coremltools/converters/nnssa/frontend/tensorflow/graph_pass/constant_propagation.py:62: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



['b1/read:0', 'lstm_fused_cell/range_1/delta:0', 'Minimum_3/y:0', 'b3:0', 'lstm_fused_cell/ExpandDims_1/dim:0', 'lstm_fused_cell/range/delta:0', 'lstm_fused_cell/transpose/perm:0', 'h6:0', 'lstm_fused_cell/zeros:0', 'lstm_fused_cell/range_1/start:0', 'zeros:0', 'lstm_fused_cell/zeros/shape_as_tensor:0', 'lstm_fused_cell/zeros/Const:0', 'h1:0', 'lstm_fused_cell/ExpandDims/dim:0', 'transpose/perm:0', 'h6/read:0', 'h5:0', 'lstm_fused_cell/range/start:0', 'b6/read:0', 'Reshape_1/shape:0', 'lstm_fused_cell/ExpandDims_2/dim:0', 'lstm_fused_cell/range_1:0', 'h3/read:0', 'b2:0', 'lstm_fused_cell/SequenceMask/ExpandDims/dim:0', 'lstm_fused_cell/SequenceMask/Const_2:0', 'b2/read:0', 'lstm_fused_cell/concat/axis:0', 'Reshape_2/shape:0', 'h5/read:0', 'lstm_fused_cell/kernel/read:0', 'lstm_fused_cell/bias/read:0', 'lstm_fused_cell/Tile/multiples:0', 'b5:0', 'b6:0', 'lstm_fused_cell/range_1/limit:0', 'b3/read:0', 'zeros/Const:0', 'Reshape/shape:0', 'b5/read:0', 'h1/read:0', 'lstm_fused_cell/bias:0',

In [5]:
# Generate some random data as inputs
input_node = np.random.rand(1, 16, 19, 26) * 10.
input_lengths = np.array([16], dtype=np.int32)
previous_state_c = np.random.rand(1, 2048)
previous_state_h = np.random.rand(1, 2048)

In [6]:
# Run predictions
out = model.predict({
    'input_node': input_node,
    'input_lengths': input_lengths,
    'previous_state_h': previous_state_h,
    'previous_state_c': previous_state_c
})['logits']

output = np.array(out)
print(output)

[[[9.53124891e-06 9.94660786e-06 4.83189033e-05 3.77420868e-08
   2.37063468e-05 2.40536410e-05 3.72992645e-07 1.31326542e-05
   1.61471871e-05 4.65087978e-05 1.07197133e-11 4.61233007e-08
   8.13365295e-06 1.67659530e-03 1.87073961e-01 1.04468199e-06
   9.51681542e-08 1.06654418e-15 1.66139962e-05 9.98478208e-05
   1.90330166e-02 1.93424032e-08 4.07183288e-05 7.66751441e-07
   5.59042992e-13 1.40265911e-05 1.25609065e-10 5.51793611e-10
   7.91843414e-01]]

 [[7.74746077e-05 2.77525292e-08 7.82753978e-06 9.32246493e-08
   3.64371454e-06 2.48085735e-06 1.74522974e-08 7.14242342e-05
   5.72747649e-06 2.26166875e-07 1.29338359e-10 3.31685635e-09
   6.21018671e-07 4.59876319e-05 1.88309365e-04 2.77767835e-08
   1.05114752e-08 9.89275952e-15 7.71202906e-08 9.49954665e-06
   3.90891312e-03 1.71199532e-09 5.89598130e-07 1.11329129e-07
   3.07641357e-10 2.99414387e-04 2.11845368e-11 3.79385456e-09
   9.95377541e-01]]

 [[1.90417038e-03 4.93111145e-07 1.04704907e-06 3.42776616e-08
   9.21732237

In [7]:
# Optionally we can verify the predictions are consistant with TensorFlow's output
from tensorflow.contrib.rnn import *

with open(tfmodel_path, 'rb') as f:
    serialized = f.read()
original_gdef = tf.compat.v1.GraphDef()
original_gdef.ParseFromString(serialized)

tf.import_graph_def(original_gdef, name="")

with tf.Session() as sess:
    g = sess.graph
    out = g.get_tensor_by_name('Softmax:0')
    in1 = g.get_tensor_by_name('input_node:0')
    in2 = g.get_tensor_by_name('input_lengths:0')
    in3 = g.get_tensor_by_name('previous_state_c:0')
    in4 = g.get_tensor_by_name('previous_state_h:0')

    tf_out = sess.run(out, feed_dict={
        in1: input_node, in2: input_lengths, in3: previous_state_c, in4: previous_state_h,
    })

tf_output = np.array(tf_out)

In [8]:
assert output.shape == tf_output.shape
np.testing.assert_almost_equal(output.flatten(), tf_output.flatten(), decimal=4)